# Accesing and querying JanusGraph database

This notebook demonstrates how to create a connection to the graph database, how to query the graph database and how to visualize graph query results.


Let's start with imports and connection to JanusGraph database:

In [1]:
from thoth.lab import obtain_location
from thoth.storages import GraphDatabase

# Instantiate and connect the JanusGraph database
adapter = GraphDatabase()
adapter.connect()

adapter.is_connected()

True

Next, let's try to perform some generic queries. We will use `GraphQueryResult` wrapper that wraps our graph queries and exposes some useful methods for us:

In [2]:
from thoth.lab import GraphQueryResult as gqr

query = adapter.g.V().has('__label__', 'python_package_version').groupCount().by('package_name').next()
query_result = gqr(query)

If we would like to access raw response, we can do so by acessing the result attribute:

In [3]:
query_result.result

{'absl-py': 11,
 'astor': 3,
 'bleach': 1,
 'chardet': 1,
 'gast': 1,
 'grpcio': 21,
 'html5lib': 26,
 'iniparse': 1,
 'kitchen': 1,
 'markdown': 4,
 'numpy': 14,
 'pip': 2,
 'protobuf': 6,
 'pyliblzma': 1,
 'pyxattr': 1,
 'setuptools': 304,
 'six': 23,
 'smartcols': 1,
 'tensorboard': 12,
 'tensorflow': 2,
 'termcolor': 1,
 'webencodings': 6,
 'werkzeug': 12,
 'wheel': 8}

We can get results as Pandas dataframe:

In [4]:
import pandas as pd

r=query_result.result
df = pd.DataFrame(list(r.items()), columns=['Package Name', '# Versions'])
df

,Package Name,# Versions
0,absl-py,11
1,protobuf,6
2,werkzeug,12
3,iniparse,1
4,six,23
5,html5lib,26
6,wheel,8
7,smartcols,1
8,astor,3
9,setuptools,304


If we get result as a dictionary of values, we can easily plot some graphs:

In [5]:
query = adapter.g.V().has('__label__', 'python_package_version').groupCount().by('package_name').next()
query_result = gqr(query)

In [6]:
query_result.plot_bar()

In [7]:
# Or a pie chart

query_result.plot_pie()

To get more options on how to visualize and interact with data retrieved from graph database, use the `help` function to access up-to-date `GraphQueryResult` documentation:

In [8]:
help(gqr)

Help on class GraphQueryResult in module thoth.lab.graph:

class GraphQueryResult(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, result)
 |      Initialization.
 |      
 |      :param result: the result to be used as a query result, can be directly coroutine that is fired.
 |  
 |  plot_bar(self)
 |      Plot histogram of results obtained.
 |  
 |  plot_pie(self)
 |      Plot a pie of results into Jupyter notebook.
 |  
 |  serialize(self)
 |      Serialize the output of graph query.
 |  
 |  to_dataframe(self)
 |      Construct a panda's dataframe on results.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



For more info on how to construct queries to the graph database, see [Practical Gremlin: An Apache TinkerPop Tutorial](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html). Also see [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame) documentation on how to use dataframes and [plotly documentation](https://plot.ly/api/) for creating interactive figures. To visualize data available in the graph, use the [graph explorer](https://url.corp.redhat.com/thoth-sbu-graphexp).